In [1]:
%%capture
!pip install --no-cache-dir "unsloth==2025.11.2"
!pip install accelerate bitsandbytes huggingface_hub transformers func-timeout

**Nhập API The Hugging Face để sử dụng**

In [2]:
from huggingface_hub import login
login()

**Import thư viện**

In [3]:
from unsloth.chat_templates import get_chat_template
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from func_timeout import func_timeout, FunctionTimedOut
from datetime import datetime
import pandas as pd
import signal
import torch
import time
import os
import re

**Load model**

In [5]:
model_name = "qwen" # Chọn "qwen" hoặc "llama3"
model_id = "unsloth/Qwen2.5-7B-Instruct"
max_seq_length = 4096
dtype = None
load_in_4bit = True  # Dùng 4bit để tiết kiệm VRAM

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_id,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Chế độ Inference (Tăng tốc độ lên 2x)
FastLanguageModel.for_inference(model)

# Setup chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
    map_eos_token = True,
)

==((====))==  Unsloth 2025.11.2: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

**Load Data**

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load test data
checkpoint_path = '/content/drive/MyDrive/T2C_base_qwen2/base_qwen2.csv'
test_path = '/content/drive/MyDrive/T2C_base_qwen2/test(2).csv'
test_df = pd.read_csv(test_path, encoding="utf-8-sig")

print(f"Loaded test shape: {test_df.shape}")

Loaded test shape: (2422, 6)


**Prompt Text2Cypher**

In [12]:
def create_messages(question, schema):
    system_message = """Task: Generate a Cypher statement to query a graph database. Instructions: Use only the provided relationship types and properties in the schema. Do not use any other relationship types or properties that are not provided in the schema. Do not include any explanations or apologies in your responses. Do not respond to any questions that ask anything other than constructing a Cypher statement. Do not include any text except the generated Cypher statement."""

    user_content = f"""Generate Cypher statement to query a graph database. Use only the provided relationship types and properties in the schema.
 Schema: {schema}
 Question: {question}
 Cypher output:"""

    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_content}
    ]

    return messages

print("Message creation function loaded successfully")

Message creation function loaded successfully


**Hàm chuyển đổi Text2Cypher**

In [13]:
def generate_cypher_raw(question, schema, timeout=30):
    messages = create_messages(question, schema)

    def _generate():
        # Apply chat template và tokenize
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
        ).to(model.device)
        # Create attention mask (very important!)
        attention_mask = (inputs != tokenizer.pad_token_id).long()
        # Generate
        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs,
                attention_mask=attention_mask,
                max_new_tokens=256,
                temperature=0.1,
                do_sample=True,
                top_p=0.9,
                use_cache=True,
                pad_token_id=tokenizer.eos_token_id,
            )

        # Decode output (giữ special tokens)
        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=False)
        return generated_text

    try:
        return func_timeout(timeout, _generate)
    except FunctionTimedOut:
        print(f"[TIMEOUT] Generation exceeded {timeout}s")
        return "time_error"
    except Exception as e:
        print(f"[ERROR] Generation failed: {e}")
        return "error"

**Hàm format câu cypher từ output**

In [14]:
MODEL_TOKENS = {
    "qwen": {"start": "<|im_start|>assistant", "end": "<|im_end|>", "special_tokens": ["<|im_end|>", "<|im_start|>", "<|eot_id|>", "<|start_header_id|>", "<|end_header_id|>", "<|endoftext|>"]},
    "llama3": {"start": "<|start_header_id|>assistant<|end_header_id|>", "end": "<|eot_id|>", "special_tokens": ["<s>", "</s>", "<pad>"]}
}

def extract_cypher_from_text(text, model_name):
    if text in ["time_error", "error"]:
        return text

    try:
        # Bước 1: Extract assistant content
        tokens = MODEL_TOKENS[model_name]
        if tokens["start"] in text:
            after_assistant = text.split(tokens["start"])[-1]
            if tokens["end"] in after_assistant:
                assistant_content = after_assistant.split(tokens["end"])[0].strip()
            else:
                assistant_content = after_assistant.strip()
        else:
            assistant_content = text.strip()

        # Bước 2: Extract code block đầu tiên (giữa ``` ```)
        match = re.search(r"```(.*?)```", assistant_content, re.DOTALL)
        if match:
            code_block = match.group(1).strip()
        else:
            code_block = assistant_content.strip()

        # Bước 3: Tìm "MATCH" và trích xuất Cypher từ đó
        code_block_lower = code_block.lower()
        match_index = code_block_lower.find("match")
        if match_index == -1:
            return "error"

        cypher = code_block[match_index:].strip()

        # Loại bỏ special tokens
        for token in tokens["special_tokens"]:
            cypher = cypher.replace(token, "")
        cypher = cypher.strip()

        return cypher
    except:
        return "error"

print("✓ extract_cypher_from_text loaded")

✓ extract_cypher_from_text loaded


In [15]:
def generate_cypher(question, schema, timeout=30, model_name=model_name):

    # Generate raw output
    raw_output = generate_cypher_raw(question, schema, timeout)

    # Extract Cypher
    cypher = extract_cypher_from_text(raw_output, model_name)

    return cypher

print("✓ generate_cypher loaded")

✓ generate_cypher loaded


**Test thử 1 dòng**

In [20]:
# Lấy test case đầu tiên
first_row = test_df.iloc[2048]
test_question = first_row['question']
test_schema = first_row['schema']

print("="*80)
print("TEST QUESTION:")
print("="*80)
print(test_question)

# ============================================================================
# Test 1: generate_cypher_raw
# ============================================================================
print("\n" + "="*80)
print("TEST 1: generate_cypher_raw()")
print("="*80)
raw_output = generate_cypher_raw(test_question, test_schema, timeout=30)
print("RAW OUTPUT:")
print(raw_output)

# ============================================================================
# Test 2: generate_cypher
# ============================================================================
print("\n" + "="*80)
print("TEST 2: generate_cypher()")
print("="*80)
final_cypher = generate_cypher(test_question, test_schema, timeout=30, model_name=model_name)
print(final_cypher)

# ============================================================================
# Test 3: EXPECTED CYPHER
# ============================================================================
print("\n" + "="*80)
print("TEST 3: EXPECTED CYPHER")
print("="*80)
if 'cypher' in test_df.columns:
    print(first_row['cypher'])
else:
    print("N/A")

TEST QUESTION:
Find the Journal with a name starting with J, and linked with an Article through PUBLISHED_IN relationship. The Article must have abstract:   Using matrix inversion and determinant evaluation techniques we prove several
summation and transformation formulas for terminating, balanced,
very-well-poised, elliptic hypergeometric series.
 and be PUBLISHED_IN with pages recorded!

TEST 1: generate_cypher_raw()
RAW OUTPUT:
<|im_start|>system
Task: Generate a Cypher statement to query a graph database. Instructions: Use only the provided relationship types and properties in the schema. Do not use any other relationship types or properties that are not provided in the schema. Do not include any explanations or apologies in your responses. Do not respond to any questions that ask anything other than constructing a Cypher statement. Do not include any text except the generated Cypher statement.<|im_end|>
<|im_start|>user
Generate Cypher statement to query a graph database. Use only

**Chạy theo checkpoint toàn file**

In [ ]:
def run_with_checkpoint(
    test_df,
    checkpoint_path,
    timeout=60,
    model_name=model_name,
    log_interval=100
):
    # ==========================================================================
    # BƯỚC 1: Kiểm tra và load checkpoint
    if os.path.exists(checkpoint_path):
        print(f"[CHECKPOINT] Tìm thấy file checkpoint: {checkpoint_path}")
        df = pd.read_csv(checkpoint_path, encoding="utf-8-sig")
        print(f"[CHECKPOINT] Đã load {len(df)} dòng từ checkpoint")

        # Đếm số dòng đã xử lý
        processed_count = df['cypher_generated'].notna().sum()
        print(f"[CHECKPOINT] Đã xử lý: {processed_count}/{len(df)} dòng")

    else:
        print(f"[CHECKPOINT] Không tìm thấy checkpoint, tạo mới từ test_df")
        df = test_df.copy()
        df['cypher_generated'] = ''  # Thêm cột cypher_generated trống

        # Lưu file checkpoint ban đầu
        df.to_csv(checkpoint_path, index=False, encoding='utf-8')
        print(f"[CHECKPOINT] Đã tạo file checkpoint: {checkpoint_path}")

    # ==========================================================================
    # BƯỚC 2: Xử lý các dòng chưa có kết quả
    total_rows = len(df)

    # Biến đếm cho log
    success_count = 0
    error_count = 0
    timeout_error_count = 0
    batch_start_idx = 0

    # Biến đếm để lưu checkpoint
    processed_since_last_save = 0

    print(f"\n{'='*80}")
    print(f"BẮT ĐẦU XỬ LÝ - Tổng số dòng: {total_rows}")
    print(f"{'='*80}\n")

    start_time = time.time()

    for idx in range(total_rows):
        # Kiểm tra nếu dòng này đã có kết quả
        current_cypher = df.at[idx, 'cypher_generated']

        # Nếu đã có dữ liệu (không phải NaN và không phải chuỗi rỗng)
        if pd.notna(current_cypher) and str(current_cypher).strip() != '':
            continue  # SKIP dòng này

        # ======================================================================
        # XỬ LÝ DÒNG CHƯA CÓ KẾT QUẢ
        print(f"[Processing] Dòng {idx}...", end=" ", flush=True)

        try:
            question = df.at[idx, 'question']
            schema = df.at[idx, 'schema']

            # Gọi hàm generate_cypher
            cypher_result = generate_cypher(question, schema, timeout=timeout, model_name=model_name)

            # Lưu kết quả
            df.at[idx, 'cypher_generated'] = cypher_result

            # Đếm theo loại kết quả và in log
            if cypher_result == "error":
                error_count += 1
                print("ERROR")
            elif cypher_result == "time_error":
                timeout_error_count += 1
                print("TIMEOUT")
            else:
                success_count += 1
                print("SUCCESS")

            # Tăng biến đếm để lưu checkpoint
            processed_since_last_save += 1

        except Exception as e:
            # Xử lý lỗi không mong đợi
            print(f"ERROR - {str(e)}")
            df.at[idx, 'cypher_generated'] = "error"
            error_count += 1
            processed_since_last_save += 1

        # ======================================================================
        # LƯU CHECKPOINT SAU MỖI 50 DÒNG
        # ======================================================================
        if processed_since_last_save >= 50:
            df.to_csv(checkpoint_path, index=False, encoding='utf-8')
            print(f"[CHECKPOINT] Đã lưu sau {processed_since_last_save} dòng")
            processed_since_last_save = 0

        # ======================================================================
        # LOG THỐNG KÊ MỖI 100 DÒNG
        # ======================================================================
        if (idx + 1) % log_interval == 0:
            elapsed_time = time.time() - start_time
            avg_time_per_row = elapsed_time / (idx + 1)
            remaining_rows = total_rows - (idx + 1)
            estimated_time = avg_time_per_row * remaining_rows

            print(f"\n{'='*80}")
            print(f"[LOG] Dòng {batch_start_idx}-{idx}")
            print(f"{'='*80}")
            print(f"Thành công:     {success_count}")
            print(f"Error:          {error_count}")
            print(f"Timeout Error:  {timeout_error_count}")
            print(f"Tổng xử lý:     {success_count + error_count + timeout_error_count}")
            print(f"Tiến độ:        {idx + 1}/{total_rows} ({(idx + 1)/total_rows*100:.2f}%)")
            print(f"Thời gian:      {elapsed_time/60:.2f} phút")
            print(f"Ước tính còn:   {estimated_time/60:.2f} phút")
            print(f"{'='*80}\n")

            # Reset bộ đếm cho batch tiếp theo
            success_count = 0
            error_count = 0
            timeout_error_count = 0
            batch_start_idx = idx + 1

    # ==========================================================================
    # LƯU CHECKPOINT CUỐI CÙNG (nếu còn dòng chưa lưu)
    # ==========================================================================
    if processed_since_last_save > 0:
        df.to_csv(checkpoint_path, index=False, encoding='utf-8')
        print(f"[CHECKPOINT] Đã lưu {processed_since_last_save} dòng cuối cùng")

    # ==========================================================================
    # KẾT THÚC - LOG CUỐI CÙNG
    # ==========================================================================
    total_time = time.time() - start_time

    # Đếm tổng kết quả
    final_success = (df['cypher_generated'].notna() &
                     (df['cypher_generated'] != 'error') &
                     (df['cypher_generated'] != 'time_error') &
                     (df['cypher_generated'] != '')).sum()
    final_error = (df['cypher_generated'] == 'error').sum()
    final_timeout = (df['cypher_generated'] == 'time_error').sum()

    print(f"\n{'='*80}")
    print(f"HOÀN THÀNH")
    print(f"{'='*80}")
    print(f"Tổng số dòng:        {total_rows}")
    print(f"Thành công:          {final_success} ({final_success/total_rows*100:.2f}%)")
    print(f"Error:               {final_error} ({final_error/total_rows*100:.2f}%)")
    print(f"Timeout Error:       {final_timeout} ({final_timeout/total_rows*100:.2f}%)")
    print(f"Tổng thời gian:      {total_time/60:.2f} phút")
    print(f"Thời gian trung bình: {total_time/total_rows:.2f} giây/dòng")
    print(f"Kết quả đã lưu tại:  {checkpoint_path}")
    print(f"{'='*80}\n")
    return df

In [ ]:
# Chạy lần đầu hoặc tiếp tục từ checkpoint
result_df = run_with_checkpoint(
    test_df=test_df,
    checkpoint_path=checkpoint_path,
    timeout=60,
    model_name=model_name,
    log_interval=100
)

[CHECKPOINT] Không tìm thấy checkpoint, tạo mới từ test_df
[CHECKPOINT] Đã tạo file checkpoint: /content/drive/MyDrive/T2C_base_qwen2/base_qwen2.csv

BẮT ĐẦU XỬ LÝ - Tổng số dòng: 2422

[Processing] Dòng 0... SUCCESS
[Processing] Dòng 1... SUCCESS
[Processing] Dòng 2... SUCCESS
[Processing] Dòng 3... SUCCESS
[Processing] Dòng 4... SUCCESS
[Processing] Dòng 5... SUCCESS
[Processing] Dòng 6... 